In [7]:
from trainlib.FileCollection import FileCollection
import trainlib.cuts as cuts
from trainlib.utils import read_data
from trainlib.RNNPreprocessor import RNNPreprocessor
from trainlib.PCAWhiteningPreprocessor import PCAWhiteningPreprocessor
from trainlib.generator import Generator
import pandas as pd
import numpy as np
import pickle

In [35]:
# read some input data
problematic_branches = ["PFMET", "nCleanedJetsPt30", "nCleanedJetsPt30BTagged_bTagSF", "nExtraLep", "D_VBF1j_ggH_ME", "nExtraZ", "ZZMass", "Z2Pt", "Z1Pt", "Z2Mass", "Z1Mass"]
fcoll = FileCollection({"/data_CMS/cms/wind/CJLST_NTuples/ggH125/ZZ4lAnalysis.root" : cuts.no_cut}, 0.0, 1.0)
setup_data = read_data(fcoll, 0, 110483, branches = problematic_branches)

skimming /data_CMS/cms/wind/CJLST_NTuples/ggH125/ZZ4lAnalysis.root
collection set up: 1 files, 110483 entries in total, 110483 of which will be used


In [36]:
setup_data.loc[setup_data["nExtraZ"] > 0]

,PFMET,nCleanedJetsPt30,nCleanedJetsPt30BTagged_bTagSF,nExtraLep,D_VBF1j_ggH_ME,nExtraZ,ZZMass,Z2Pt,Z1Pt,Z2Mass,Z1Mass
2489,18.929331,0,0,2,0.261911,1,104.014984,12.784993,19.666384,45.382992,49.726887
49924,40.288765,1,0,2,0.233332,1,107.638596,13.859759,13.049999,31.234056,73.151329
51835,34.429962,1,0,2,0.566255,1,96.390297,23.187113,14.606493,27.719847,61.321182
53342,33.510994,0,0,2,0.266659,1,102.064476,24.286678,31.529131,23.513140,62.091290
79412,51.428452,0,0,2,0.321529,1,84.214584,21.754971,22.419390,28.165222,55.104885
80396,26.268425,0,0,2,0.381637,1,70.739967,27.180670,23.727049,20.087019,43.330177
88164,17.591505,0,0,2,0.292827,1,92.600426,19.198095,20.065569,16.023314,48.548168


In [37]:
H1_stream = {"/data_CMS/cms/wind/CJLST_NTuples/ggH125/ZZ4lAnalysis.root" : cuts.no_cut}
H0_stream = {"/data_CMS/cms/wind/CJLST_NTuples/VBFH125/ZZ4lAnalysis.root" : cuts.no_cut}

In [38]:
testgen = Generator(H1_stream, H0_stream, problematic_branches, preprocessor = None)
setup_len = testgen.setup_training_data()

skimming /data_CMS/cms/wind/CJLST_NTuples/ggH125/ZZ4lAnalysis.root
collection set up: 1 files, 110483 entries in total, 55241 of which will be used
skimming /data_CMS/cms/wind/CJLST_NTuples/VBFH125/ZZ4lAnalysis.root
collection set up: 1 files, 62320 entries in total, 31160 of which will be used


In [39]:
preprocessor_cuts = lambda row: row["nCleanedJetsPt30"] >= 2

In [40]:
pre = PCAWhiteningPreprocessor("test", problematic_branches, preprocessor_cuts)

PCA setup for 'test': ['PFMET', 'nCleanedJetsPt30', 'nCleanedJetsPt30BTagged_bTagSF', 'nExtraLep', 'D_VBF1j_ggH_ME', 'nExtraZ', 'ZZMass', 'Z2Pt', 'Z1Pt', 'Z2Mass', 'Z1Mass']
PCAWhiteningPreprocessor for stream 'test'


In [41]:
pre.setup_generator(testgen.raw_generator_scrambled(), setup_len)

H1 contains 55241 entries
H0 contains 31160 entries
using the following chunk sizes: (55 / 31)
setting up PCA whitening on 86430 events
19918 remaining after the cuts


In [42]:
pre.process(setup_data)

{'test': array([[  2.30983671e+00,  -1.26247158e+00,  -1.44385783e+00, ...,
          -1.02951791e-01,  -1.22423263e-02,   4.15094331e+01],
        [ -3.43519848e-01,  -2.95291665e-01,  -9.02323526e-02, ...,
          -1.67443400e-01,  -2.57137664e-02,  -4.64163200e+01],
        [  2.23862188e+00,  -1.70558304e+00,  -2.09613997e-01, ...,
          -1.02897377e-03,   1.97158497e-02,  -5.62476137e+01],
        ..., 
        [  4.24689891e-01,  -6.61172561e-01,   1.29456692e+00, ...,
          -3.12980190e-02,   2.06888308e-02,   1.15896915e+01],
        [ -4.98463425e-01,   5.93281877e-01,   5.52779421e-01, ...,
          -2.63092789e+00,  -3.15417862e-02,  -1.28245814e+01],
        [  1.57354251e-01,  -9.45683393e-01,   4.13396244e-01, ...,
          -2.34302900e-02,  -1.52603263e-01,   3.44694155e+01]])}

In [43]:
pre.pca.explained_variance_

array([  3.85206711e+03,   7.77997803e+02,   4.10884837e+02,
         1.59017983e+02,   6.13043227e+01,   2.84195011e+01,
         3.38753914e-01,   6.02155527e-02,   7.65220815e-03,
         2.04385124e-03,   2.02222726e-34])